(Sim_Accel)=
# Improving Sampling and Accelerating Simulations

In many cases of interest, carrying out robust free energy calculations may require significant investment of computational resources, beyond that which can be obtained by most researchers. This page examines additional tools for accelerating the sampling. As these begin to branch out from the [fundamentals of free energy](fundamentals), we will not go deeply into all these methods. They are not needed to carry out free energy calculations, but they may be extremely useful to converge calculations in complex systems with slow dynamics and we encourage readers to examine these topics to see what will work for them.

## Umbrella Sampling
One standard method for improving sampling in atomistic simulations is umbrella sampling,{{cite|Torrie1977}} where bias terms are added to constrain the simulation in some way, then the restraints are removed. This method can lower barriers on the potential energy surface, or restrain simulations to slow-interconverting configurations that are relevant to the binding affinities (e.g. different torsional states). Doing this allows for the free energy components to be properly computed and then combined.{{cite|Wang2006}}{{cite|Mobley2007a}}{{cite|Mobley2007b}}
If a system has slow degrees of freedom, like some hydration free energies,{{cite|Klimovich2010}} this method allows one to sample more frequently in the slow state.

One can also envision using umbrella sampling to compute the free energy of constraining the free ligand into the bound conformation directly before computing the free energy of binding, and then computing the free energy of releasing these restraints in solution. Doing so usually decreases correlation times for sampling intermediate states, thereby increasing the simulation efficiency. {{cite|Wang2006}}{{cite|Woo2005}}

## Hamiltonian Replica Exchange
Although it is perfectly valid to run each intermediate state as its own simulation, there are tools that let you run parallel, coupled simulations and swap information between them to improve sampling. One such method is called Hamiltonian Exchange (HEX) or Hamiltonian Replica Exchange (HREX). In HEX, the parallel simulations at each alchemical variable are allowed to swap atoms/molecules (under certain conditions) from state A which has different energy barriers than state B. This enables sampling configurations that may take significant amounts of time to access normally when simulating only one state.
{{cite|Okamoto2004}}{{cite|Roux2007}}{{cite|Woods2003}}{{cite|Banba2000}}{{Cite|Bitetti-Putzer2003|}}{{cite|Hritz2008}}

We **highly recommend doing HEX methods** as most codes do this on top of well-tested temperature replica exchange methods, and so there is very little overhead. the outputs of Hamiltonian exchange simulations can be analyzed in the same way as the outputs of $K$ uncoupled simulations. These simulations are guaranteed to decorrelate as fast or faster than standard simulations, though the exact amount of improvement depends on the system; at least they will not be any worse!

[Correlation](correlation) times with HEX is a bit more complicated since atoms/molecules are swapping along very different trajectories. In this case, you will need to compute along trajectories that sample different states as opposed to a single state.

### Some Suggestions for Replica Exchange Simulations

* The level of acceleration one gets is a function both of phase space mixing between the replicas and the acceleration provided by of replicas kinetics that have faster sampling than the original distribution in the degree of freedom of interest. 
* Choice of Hamiltonian/temperature states depends quite a bit on what one actually wants to do: is it just accelerating kinetics, or does one actually care about properties as a function of temperature or calculating free energies? That can change the choice.  Identifying a choice of replica that accelerates kinetics in desired way is nontrivial problem. 
* Space replicas to be even in phase space overlap.  The easiest way to do this is to guess a spacing, then look at the results pf MBAR /TI after a short run. You can then respace the replicas so the error in the free energies between consecutive states is close to being equal. It is most important to get rid of very bad spacing; there is not much efficiency gain in going from decent spacing to optimal spacing {{cite|Naden2014}}. 
* Acceptance rates should be around 38% to maximize mixing in state space, but anywhere in the 25-50% range is nearly as good, so it is not that useful to tune it too much {{cite|Predescu2005}}.
    * More precise quantitative measurements of state space mixing are determined by taking the transition matrix between states, and maximizing $1-\lambda_2$, where $\lambda_2$ is the second largest eigenvalue of the transition matrix. {{cite|Chodera2011}}  This quantity is known as the spectral gap, and is related to the rate of mixing in state space.  Remember that mixing in state space is insufficient to accelerate sampling; there must be some state where configurations are sampled more quickly. 
* To improve mixing between states, perform multiple replica switches each time one pauses to exchange. {{cite|Chodera2011}}  Anecdotal evidence is that effectiveness of multiple switches decreases past 10-100 exchanges, but this is system dependent.
* Exchange should be as frequent as is permissible given the networking and code {{cite|Chodera2011}}, as long as the velocities are not reassigned on switches.   Reassigning velocities too frequently slows kinetics and negates the advantages gained by performing rapid replica switches {{cite|Basconi2013}}.

## Other Methods
The other methods that will be mentioned briefly are *Expanded Ensemble* and *$\lambda$-dynamics.* Expanded ensemble works by running a single simulation and sampling both the intermediate states and separate coordinates simultaneously. $\lambda$-dynamics works by treating the alchemical variable as dynamical, introducing a fictitious mass corresponding to the $\lambda$ degree of freedom; although this seems rather novel, it is essentially identical to Monte Carlo techniques.{{cite|Banba2000}}{{cite|Guo1998}}{{cite|Kong1996}}{{cite|Li2007}}
$\lambda$-dynamics is showing promise, but is still in preliminary stages of development{{cite|Zheng2008a}}{{cite|Zheng2008b}}{{cite|Min2008}}{{cite|Li2007}}{{cite|Min2007}}

In general, we **do NOT recommend Expanded Ensemble or $\lambda$-dynamics to beginners.** The methods and implementations are not up to the same robustness as HEX yet and there are tweaks and extra parameters that have to be coded to obtain proper convergence. Given more time and development, these methods may become more accessible to beginners and we will be able to recommend them in the future.

### Slow Growth 

In slow-growth methods, the coupling parameter, $\lambda$, is slowly varied over the course of one or several simulations to take a system gradually from $\lambda=0$ to $\lambda=1$; from this, the free energy difference between endpoints is estimated. In equilibrium methods, on the other hand, separate simulations are run at multiple different lambda values and information from the individual simulations is used to estimate free energy differences between adjoining lambda values.

Slow-growth methods have a number of well-documented problems, such as Hamiltonian lag and hysteresis (*add references*). Additionally, if a slow-growth calculation does not lead to a sufficiently precise free energy estimate, the only way to improve the free energy estimate is to repeat the calculation with a slower rate of change in lambda – the simulation cannot be extended to gain additional precision, meaning that significant data can be wasted. Fast-growth methods, while conceptually appealing, do not appear to offer substantial advantages over equilibrium methods.{{cite|Jarzynski2006}}{{cite|Oostenbrink2006}}

### Nonequilibrium (Fast Growth) methods 

Fast-growth methods are based on a theorem by Jarzynski in 1997{{cite|Jarzynski1997}} that the free energy difference associated with a particular equilibrium process can be computed by taking an the exponential average of the irreversible work done in performing many (potentially rapid) non-equilibrium trials of the same process. 

$ \Delta G = -k_B T \ln \left \langle e^{-W/k_b T} \right \rangle $

Where $ W$ is the work to execute the transition between the initial and final thermodynamic state. When applied to alchemical free energy calculations, this simply amounts to estimating free energy differences by performing many different rapid versions of a slow-growth trial – that is, rapidly changing lambda between two values (i.e. 0 and 1)  in many separate trials, and monitoring the work done each time.  Equilibrium free energy calculations can be thought of as "instantaneous growth" as they rely on estimating the free energy difference between neighboring $\lambda$ values based only on instantaneous evaluations of potential energy differences between $\lambda$ values (or by evaluation of and extrapolation of $dV/d\lambda$ at a particular $\lambda$ value).

## References
<references>
{{cite|Torrie1977|Torrie, G. M., and Valleau, J. P. (1977) Non-physical Sampling Distributions in Monte-Carlo Free-Energy Estimation : Umbrella Sampling. *J. Comput. Phys.* 23, 187–199.|http://www.citeulike.org/group/14929/article/9052620}}

{{cite|Wang2006|Wang, J., Deng, Y., and Roux, B. (2006) Absolute Binding Free Energy Calculations Using Molecular Dynamics Simulations with Restraining Potentials. *Biophys. J.* 91, 2798–2814.|http://www.citeulike.org/user/kupopo/article/867499}}

{{cite|Mobley2007a|Mobley, D. L., Chodera, J. D., and Dill, K. A. (2007) Confine-and-release method: Obtaining correct binding free energies in the presence of protein conformational change. *J. Chem. Theory Comput.* 3, 1231–1235.|http://www.citeulike.org/group/14929/article/9052442}}

{{cite|Mobley2007b|Mobley, D. L., Graves, A. P., Chodera, J. D., McReynolds, A. C., Shoichet, B. K., and Dill, K. A. (2007) Predicting absolute ligand binding free energies to a simple model site. *J. Mol. Biol.* 371, 1118–1134.\http://www.citeulike.org/group/14929/article/9052443}}

{{cite|Klimovich2010|Klimovich, P. V., and Mobley, D. L. (2010) Predicting hydration free energies using all-atom molecular dynamics simulations and multiple starting conformations. *J. Comp. Aided Mol. Design* 24, 307–316.|http://www.citeulike.org/group/14929/article/9520307}}

{{cite|Woo2005|Woo, H.-J., and Roux, B. (2005) Calculation of absolute protein-ligand binding free energy from computer simulation. *Proc. Natl. Acad. Sci.* 102, 6825–6830.|http://www.citeulike.org/group/14929/article/9052663}}

{{cite|Okamoto2004|Okamoto, Y. (2004) Generalized-ensemble algorithms: Enhanced sampling techniques for Monte Carlo and molecular dynamics simulations. *J. Mol. Graph. Model.* 22, 425–439.|http://www.citeulike.org/group/14929/article/9052463}}

{{cite|Roux2007|Roux, B., and Faraldo-Gómez, J. D. (2007) Characterization of conformational equilibria through Hamiltonian and temperature replica- exchange simulations: Assessing entropic and environmental effects. *J. Comput. Chem.* 28, 1634–1647.|http://www.citeulike.org/group/14929/article/9052542}}

{{cite|Woods2003|Woods, C. J., Essex, J. W., and King, M. A. (2003) Enhanced Configurational Sampling in Binding Free Energy Calculations. *J. Phys. Chem. B* 107, 13711–13718.|http://www.citeulike.org/group/14929/article/9052666}}

{{cite|Banba2000|Banba, S., Guo, Z., and Brooks III, C. L. (2000) Efficient sampling of ligand orientations and conformations in free energy calculations using the lambda-dynamics method. *J. Phys. Chem. B* 104, 6903–6910.|http://www.citeulike.org/group/14929/article/9052063}}

{{Cite|Bitetti-Putzer2003|Bitetti-Putzer, R., Yang, W., and Karplus, M. (2003) Generalized ensembles serve to improve the convergence of free energy simulations. *Chem. Phys. Lett.* 377, 633–641.|http://www.citeulike.org/group/14929/article/9052089}}

{{cite|Hritz2008|Hritz, J., and Oostenbrink, C. (2008) Hamiltonian replica exchange molecular dynamics using soft-core interactions. *J. Chem. Phys.* 128, 144121.|http://www.citeulike.org/group/14929/article/9052272}}

{{cite|Chodera2011| Chodera, J.D. and Shirts, M. R. (2011) Replica exchange and expanded ensemble simulations as Gibbs sampling: Simple improvements for enhanced mixing. *J. Chem. Phys.* 135, 194110 | http://www.citeulike.org/group/14929/article/10064105}}

{{cite|Basconi2013|Basconi, J. E. and Shirts, M. R. (2013) Effects of Temperature Control Algorithms on Transport Properties and Kinetics in Molecular Dynamics Simulations. *J. Chem. Theory Comput.* 9,  2887-2899 | http://www.citeulike.org/group/14929/article/12397351}}

{{cite|Naden2014|Naden, L. N., Pham T. T., and Shirts, M. R. (2014) Linear Basis Function Approach to Efficient Alchemical Free Energy Calculations. 1. Removal of Uncharged Atomic Sites. *J. Chem. Theory Comput." 10, 1128-1149 | http://www.citeulike.org/group/14929/article/13073244}}

{{cite|Predescu2005|Predescu, C., Predescu, M., Ciobanu, C. V. (2005) On the Efficiency of Exchange in Parallel Tempering Monte Carlo Simulations
*J. Phys. Chem. B, 109*, 4189-4196 |  http://www.citeulike.org/group/14929/article/875546}}

{{cite|Guo1998|Guo, Z., Brooks III, C. L., and Kong, X. (1998) Efficient and flexible algorithm for free energy calculations using the $\lambda$-dynamics approach. *J. Phys. Chem. B* 102, 2032–2036.|http://www.citeulike.org/group/14929/article/9052235}}

{{cite|Kong1996|Kong, X., and Brooks III, C. L. (1996) $\lambda$-dynamics: A new approach to free energy calculations. *J. Chem. Phys.* 105, 2414–2423.|http://www.citeulike.org/group/14929/article/9052345}}

{{cite|Li2007|Li, H., Fajer, M., and Yang, W. (2007) Simulated scaling method for localized enhanced sampling and simultaneous "alchemical" free energy simulations: A general method for molecular mechanical, quantum mechanical, and quantum mechanical/molecular mechanical simulations. *J. Chem. Phys.* 126, 024106.|http://www.citeulike.org/group/14929/article/9052376}}

{{cite|Zheng2008a|Zheng, L., Carbone, I. O., Lugovskoy, A., Berg, B. A., and Yang, W. (2008) A hybrid recursion method to robustly ensure convergence efficiencies in the simulated scaling based free energy simulations. *J. Chem. Phys.* 129, 034105.|http://www.citeulike.org/user/nathanandrewbaker/article/6361266}}

{{cite|Zheng2008b|Zheng, L., and Yang, W. (2008) Essential energy space random walks to accelerate molecular dynamics simulations: Convergence improvements via an adaptive-length selfhealing strategy. *J. Chem. Phys.* 129, 014105.}}

{{cite|Min2008|Min, D., and Yang, W. (2008) Energy difference space random walk to achieve fast free energy calculations. *J. Chem. Phys.* 128, 191102.}}

{{cite|Li2007|Li, H., and Yang, W. (2007) Forging the missing link in free energy estimations: lambda-WHAM in thermodynamic integration, overlap histogramming, and free energy perturbation. *Chem. Phys. Lett.* 440, 155–159.|http://www.citeulike.org/group/14929/article/9052374}}

{{cite|Min2007|Min, D., Li, H., Li, G., Bitetti-Putzer, R., and Yang, W. (2007) Synergistic approach to improve “alchemical” free energy calculation in rugged energy surface. *J. Chem. Phys.* 126, 144109.|http://www.citeulike.org/group/14929/article/9052435}}

{{cite|Jarzynski1997|Jarzynski, C. Physical Review Letters 1997, 78.|http://www.citeulike.org/group/14929/article/9052290}}

{{cite|Jarzynski2006|Jarzynski, C. Phys. Rev. E 2006, 73, 046105.|http://www.citeulike.org/group/14929/article/9052297}}

{{cite|Oostenbrink2006|Oostenbrink, C.;  van Gunsteren, W. F. Chemical Physics 2006, 323.|http://www.citeulike.org/group/14929/article/9052473}}
</references>